In [1]:
# you need to install paho client for this to work and you should have a running
# mqtt broker on your local system. 
#pip install paho-mqtt

In [2]:
# mqtt client
import paho.mqtt.client as paho
import syft as sy

In [3]:
class PahoClient():

    def __init__(self, host="localhost", port=1883):
        self.host = host
        self.port = port
        self.client = paho.Client()
        self.client.on_subscribe = self.on_subscribe
        self.client.on_message = self.on_message
        self.client.on_publish = self.on_publish
        self.client.on_log = self.on_log

    def on_log(self, client, obj, level, string):
        print(string)

    def on_publish(self, client, userdata, mid):
        pass
    
    def on_subscribe(self, client, userdata, mid, granted_qos):
        print("Subscribed: "+str(mid)+" "+str(granted_qos))

    def on_message(self, client, userdata, msg):
        print(msg.topic+" "+str(msg.qos)+" "+str(len(msg.payload)))
        #print("size of message is: "+ str(sys.getsizeof(msg.payload)))
        return self.processor(msg = sy.ReprMessage(msg.payload))

    def send(self, topic, msg):
        (rc, mid) = self.client.publish(topic, str(msg), qos=1, retain=False)

    def connect(self):
        self.client.connect(self.host, self.port)

    def listen(self, topic, processor, background = True):
        
        self.processor = processor
        
        self.client.subscribe(topic, qos=1)
        if not background:
            self.client.loop_forever()
        else:
            self.client.loop_start()


In [4]:
class BroadcastConnection(PahoClient, sy.core.io.connection.ClientConnection): 
         def send_msg_with_reply(self, topic, msg): 
             if not self.client.is_connected(): 
                 self.connect() 
             self.send(topic, msg) 
         def send_msg_without_reply(self, topic, msg): 
             if not self.client.is_connected(): 
                 self.connect() 
             self.send(topic, msg) 
              


In [5]:
conn = BroadcastConnection()

In [6]:
conn.send_msg_without_reply('test', 'hello mqtt!')   

Sending CONNECT (u0, p0, wr0, wq0, wf0, c1, k60) client_id=b''
Sending PUBLISH (d0, q1, r0, m1), 'b'test'', ... (11 bytes)


If you had a mosquito client listening in your system, you'd hear the message!

you can run the client like this, after you've installed it {insert installation link}
> mosquitto_sub -t "test"


now let's build a network and routes

In [7]:
vm = sy.VirtualMachine(name = "tf-cuda-vm")
device = sy.Device(name = "home-pc")
domain = sy.Domain(name = "Lisa")
network = sy.Network(name = "science-team")

In [8]:
domain.id

<UID:8f56d176-05e2-4238-8170-09e60e5cfa6a>

In [9]:
# we are going to make them all talk over the pub-sub route
broadcast_route_network_domain = sy.core.io.route.BroadcastRoute(topic = 'science-team', connection = conn)
broadcast_route_domain_device = sy.core.io.route.BroadcastRoute(topic = 'lisa-devices', connection = conn)
broadcast_route_device_vm = sy.core.io.route.BroadcastRoute(topic = 'home-pc', connection = conn)

In [10]:
network_client = network.get_client(routes=[broadcast_route_network_domain])

In [11]:
network_client.send_msg_without_reply(msg = sy.ReprMessage(address=network.address))

Sending CONNECT (u0, p0, wr0, wq0, wf0, c1, k60) client_id=b''
Sending PUBLISH (d0, q1, r0, m2), 'b'science-team'', ... (82 bytes)


let's run a domain client that can listen on messages coming from the network and can broadcast to devices

In [12]:
domain_client = domain.get_client(routes=[broadcast_route_network_domain, broadcast_route_domain_device])

In [13]:
domain_client.routes

[science-team, lisa-devices]

In [14]:
domain.listen_on_route(route = broadcast_route_network_domain)

Sending SUBSCRIBE (d0, m3) [(b'science-team', 1)]
Received CONNACK (0, 0)
Sending PUBLISH (d1, q1, r0, m1), 'b'test'', ... (11 bytes)
Received PUBACK (Mid: 2)
Received SUBACK
Subscribed: 3 (1,)


In [15]:
network_client.send_msg_without_reply(msg = sy.ReprMessage(address=network.address))

Received PUBACK (Mid: 1)
Sending PUBLISH (d0, q1, r0, m4), 'b'science-team'', ... (82 bytes)
Received PUBACK (Mid: 4)
Received PUBLISH (d0, q1, r0, m1), 'science-team', ...  (82 bytes)
Sending PUBACK (Mid: 1)
science-team 1 82
got it
Caught exception in on_message: 'bytes' object has no attribute 'pub_address'
